In [1]:
import praw
from praw.models import MoreComments
import pprint
import emoji
import regex
from textblob import TextBlob
from textblob_nl import PatternTagger, PatternAnalyzer
from prawcore.exceptions import Forbidden
import pandas as pd

In [2]:
# Initialise the API python wrapper
reddit = praw.Reddit(client_id='HeUHguuE1BBilA', 
                     client_secret='Qc0xkQ1_Z4calpnL1Vx9k6i-2wCQTg', 
                     user_agent='Jasper Nieuwdorp',
                    username = 'jaspernieuwdorp',
                    password = 't6AD97359K')

Version 7.1.0 of praw is outdated. Version 7.2.0 was released Wednesday February 24, 2021.


In [3]:
# These sub-reddits are scraped (see also: Appendix)
nederlandse_subreddits = [
    'cirkeltrek', 'Geschiedenis', 'papgrappen', 'RMTK', 'D66', 'Politiek'
]

nederlandse_subreddits_ronde2 = [
    'coronanetherlands', 'eindhoven', 'roermond', 'tokkiefeesboek', 'nederlands', 'VeganNL'
]

In [4]:
def split_count(text):
    '''
    The idea was to extract also emojis from the posts and comments. 
    I thought it might be interesting to do something with that as well, but time prohibited me.
    '''
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [5]:
def scrape_reddit(subreddits, limit):
    '''
    Given a list of subreddits, and given a limit of the amounts of posts,
    scrape the posts of the subreddit, and its comments.
    '''
    dataframe = []

    for sub in subreddits:
        try:
            hot_posts = reddit.subreddit(sub).hot(limit = limit)
            for post in hot_posts:
                available_attributes = vars(post)
                timestamp = post.created_utc if 'created_utc' in available_attributes else None
                title = post.title if 'title' in available_attributes else None
                text = post.selftext if 'selftext' in available_attributes else None

                if text is not None and len(text) > 0:
                    # Find the emojis in a post
                    emoji_lst = split_count(text)
                    if len(emoji_lst) > 0:
                        emojis = [emoji for emoji in emoji_lst]
                    else:
                        emojis = []
                    # Calculate the TextBlob sentiment scores
                    blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
                    text_sentiment = blob.sentiment
                else:
                    emojis = []
                    text_sentiment = None

                d = {
                    'timestamp':timestamp,
                    'title':title,
                    'text':text.strip().replace('\n', ' '),
                    'sentiment':text_sentiment,
                    'emojis': emojis,
                    'ups': post.ups,
                    'downs': post.downs
                }

                dataframe.append(d)

                for comment in post.comments:
                    post.comments.replace_more(limit=None)
                    for comment in post.comments.list():
                        available_attributes = vars(comment)
                        timestamp = comment.created_utc if 'created_utc' in available_attributes else None
                        title = None
                        text = comment.body

                        if text is not None and len(text) > 0:
                            # Find the emojis in a post
                            emoji_lst = split_count(text)
                            if len(emoji_lst) > 0:
                                emojis = [emoji for emoji in emoji_lst]

                                # Calculate the TextBlob sentiment scores
                                blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
                                text_sentiment = blob.sentiment
                            else:
                                emojis = []
                                text_sentiment = None
                        d = {
                            'timestamp':timestamp,
                            'title':title,
                            'text':text.strip().replace('\n', ' '),
                            'sentiment':text_sentiment,
                            'emojis': emojis,
                            'ups': comment.ups,
                            'downs': comment.downs
                        }

                        dataframe.append(d)
        except Forbidden:
            print(Forbidden)
            print(f"We're not allowed to acces subreddit: {sub}")

    return pd.DataFrame(dataframe)

In [8]:
scrape_reddit(nederlandse_subreddits_ronde2, 10000).to_csv('reddit-results.csv')